## Filling Demand while Optimizing Cost

In cells 1-6, we import packages and read in the files we'll use. Refer to the assignment instructions for a description of the files. Note that we drop any null columns/rows.

In [2]:
import pandas as pd
import numpy as np
import pulp

In [3]:
orders = pd.read_excel('Supply chain logisitcs problem.xlsx')
orders.dropna(axis = 1, how = 'all', inplace = True)
orders.dropna(axis = 0, how = 'all', inplace = True)
orders.head()

,Order ID,Order Date,Product ID,Destination Port,Unit quantity,Weight
0,782912,2019-12-31,1700106,PORT09,808,14.30
1,782913,2019-12-31,1700106,PORT09,3188,87.94
2,782914,2019-12-31,1700106,PORT09,2331,61.20
3,782915,2019-12-31,1700106,PORT09,847,16.16
4,782916,2019-12-31,1700106,PORT09,2163,52.34


In [4]:
freight_rates = pd.read_excel('Supply chain logisitcs problem.xlsx', sheet_name  = 1)
freight_rates.dropna(axis = 1, how = 'all', inplace = True)
freight_rates.dropna(axis = 0, how = 'all', inplace = True)
freight_rates.head()

,orig_port_cd,dest_port_cd,rate
0,PORT02,PORT09,0.0484
1,PORT03,PORT09,0.1156
2,PORT04,PORT09,0.0620
3,PORT05,PORT09,0.1008
4,PORT06,PORT09,0.0848


In [5]:
wh_capacities = pd.read_excel('Supply chain logisitcs problem.xlsx', sheet_name  = 2)
wh_capacities.dropna(axis = 1, how = 'all', inplace = True)
wh_capacities.dropna(axis = 0, how = 'all', inplace = True)
wh_capacities.head()

,Plant ID,Daily Capacity
0,PLANT15,1872
1,PLANT17,731
2,PLANT18,682
3,PLANT05,531
4,PLANT02,435


In [6]:
products_per_plant = pd.read_excel('Supply chain logisitcs problem.xlsx', sheet_name  = 3)
products_per_plant.dropna(axis = 1, how = 'all', inplace = True)
products_per_plant.dropna(axis = 0, how = 'all', inplace = True)
products_per_plant.head()

,Plant Code,Product ID,Cost per unit
0,PLANT15,1698815,1.70
1,PLANT17,1664419,0.44
2,PLANT17,1664426,0.37
3,PLANT17,1672826,1.90
4,PLANT17,1674916,0.28


In [7]:
ports = pd.read_excel('Supply chain logisitcs problem.xlsx', sheet_name  = 4)
ports.dropna(axis = 1, how = 'all', inplace = True)
ports.dropna(axis = 0, how = 'all', inplace = True)
ports.head()

,Plant Code,Port
0,PLANT01,PORT01
1,PLANT01,PORT02
2,PLANT02,PORT03
3,PLANT03,PORT04
4,PLANT04,PORT05


We need to turn the shipping costs into a dictionary for easy lookup. We use the 'dict(zip(column1, column2))' paradigm.

In [8]:
shipping_costs = dict(zip(freight_rates['orig_port_cd'], freight_rates['rate']))

Next, we create a list of all unique products per plant. For now, you can treat the 'tuple' data type as a list.

In [9]:
def get_plants(product_id):
    
    temp = products_per_plant[products_per_plant['Product ID'] == product_id]
    return tuple(np.unique(temp['Plant Code']))

## Part 1 (20 points)

a) 10 points

b) 10 points

In [10]:
''' 
a) Create a new column in the 'orders' dataframe called 'allowed_plants'. 
To do this, you'll need to apply the defined get_plants function using a lambda function.
'''

# your code here
orders['allowed_plants'] = orders['Product ID'].apply(lambda x: get_plants(x))
orders

,Order ID,Order Date,Product ID,Destination Port,Unit quantity,Weight,allowed_plants
0,782912,2019-12-31,1700106,PORT09,808,14.300000,"(PLANT16,)"
1,782913,2019-12-31,1700106,PORT09,3188,87.940000,"(PLANT16,)"
2,782914,2019-12-31,1700106,PORT09,2331,61.200000,"(PLANT16,)"
3,782915,2019-12-31,1700106,PORT09,847,16.160000,"(PLANT16,)"
4,782916,2019-12-31,1700106,PORT09,2163,52.340000,"(PLANT16,)"
...,...,...,...,...,...,...,...
9210,792122,2019-12-31,1683388,PORT09,339,2.354118,"(PLANT03,)"
9211,792123,2019-12-31,1683388,PORT09,339,2.354118,"(PLANT03,)"
9212,792124,2019-12-31,1683388,PORT09,245,0.294265,"(PLANT03,)"
9213,792125,2019-12-31,1683430,PORT09,278,2.480000,"(PLANT03,)"


In [ ]:
''' 
End of Section
'''

In [11]:
'''
b) Set the index of the 'orders' dataframe to be the 'Order ID'. Make sure you set the index in place. 
'''
# your code here
orders.set_index('Order ID', inplace=True)

In [ ]:
''' 
End of Section
'''

Next, we create a dictionary to connect plants (warehouses) with the associated ports. Again, we use the 'dict(zip(column1, column2))' paradigm.

In [12]:
plant_ports = dict(zip(ports['Plant Code'], ports['Port']))

## Part 2 (60 points)

a) 25 points

b) 25 points

c) 10 points

In [13]:
def production_cost(order_id, plant):
    '''
    a) Return the production cost for a given order_id and plant (wahrehouse) name. 
    From the order id, you should first get the associated product id, which can be used to get the cost per unit.
    From here, multiply the cost per unit by the unit quantity to get the total production cost.
    '''
    
    # your code here
    t = orders.loc[order_id]
    prod_id = t['Product ID']
    pt = products_per_plant[products_per_plant['Product ID'] == prod_id]
    pt = pt[pt['Plant Code'] == plant]
    cpu = pt['Cost per unit']
    production_cost = cpu * t['Unit quantity']
    return production_cost.iloc[0]

In [ ]:
'''
End of Section
'''

In [14]:
def shipping_cost(order_id, plant):
    '''
    b) Return the shipping cost for a given order_id and plant (warehouse) name. 
    From the plant name, you should first get the associated port, which can be used to get the shipping cost per lb.
    From here, multiply the cost per lb by the weight to get the total shipping cost.
    '''
    
    # your code here
    t = orders.loc[order_id]
    w = t['Weight']
    port = plant_ports[plant]
    cp = shipping_costs[port]
    ship_cost = cp * w
    return ship_cost

In [ ]:
'''
End of Section
'''

In [15]:
def total_cost(order_id, plant):
    '''
    c) Return the total cost for a given order_id and plant (warehouse) name. 
    You should add the results of the two functions above. 
    '''
    # your code here
    return shipping_cost(order_id, plant) + production_cost(order_id, plant)

In [16]:
print(total_cost)

<function total_cost at 0x7fb3773248c0>


In [ ]:
'''
End of Section
'''

In [17]:
### We create a dictionary with the key-value pair 'orderId_plantName': total_cost.

order_costs = {}
for name, row in orders.iterrows():
    order_id = name
    for plant in row['allowed_plants']:   
        order_costs[str(order_id) + '_' + str(plant)] = total_cost(order_id, plant)

In [18]:
### We create a dictionary with the key-value pair 'plantName': list_of_orders.

plants = np.unique(ports['Plant Code'])

plant_orders = {}
for plant in plants:
    temp_list = []
    for name, row in orders.iterrows():
        if plant in row['allowed_plants']:  
            temp_list.append(str(name) + '_' + plant)
    plant_orders[plant] = temp_list

In [19]:
### We create a dictionary with the key-value pair 'plantName': capacity.

plant_cap = dict(zip(wh_capacities['Plant ID'], wh_capacities['Daily Capacity']))

In [20]:
### We create a dictionary with the key-value pair 'orderID': orderID_plantName.

order_plants = {}
temp_dict = dict(zip(orders.index, orders['allowed_plants']))
for key in temp_dict:
    temp_list = []
    for pl in temp_dict[key]:
        temp_list.append(str(key) + '_' + pl)
    order_plants[key] = temp_list

### Creating linear programming constraints

In this section, we build the linear programming problem and solve.

In [21]:
build = pulp.LpVariable.dicts("Route",order_costs.keys(),0,None, pulp.LpInteger)
prob = pulp.LpProblem("Problem",pulp.LpMinimize)
prob += pulp.lpSum([build[b] * order_costs[b] for b in order_costs.keys()]), "Total Cost"

In [22]:
for plant in plant_orders:
    if len(plant_orders[plant]) > 0:
        prob += pulp.lpSum(build[p] for p in plant_orders[plant]) <= plant_cap[plant], "Total orders out of plant %s"%plant

In [23]:
for o in order_plants:
    prob += pulp.lpSum(build[p] for p in order_plants[o]) == 1, "Order_" + str(o) + "_filled"

## Part 3 (20 points)
a) 10 pts

c) 10 pts

In [24]:
''' 
a) Solve the linear programming problem and store its status in a variable called 'status'.
'''

# your code here
status = pulp.LpStatus[prob.status]
for i in range(3):
    if status == 'Optimal':
        break
    prob.solve()

print("Status:", status)

Status: Not Solved


In [ ]:
'''
End of Section
'''

In [25]:
'''
b) Find the total cost to produce and ship all products and store the answer in a variable called total_cost
Round the final answer to 2 decimal places (https://docs.python.org/3/library/functions.html#round).
'''
# your code here
total_cost = round(pulp.value(prob.objective), 2)

print("Total Cost = ", str(total_cost))

Total Cost =  27140675.14


In [ ]:
'''
End of Section
'''